In [1]:
import os
def create_folder(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [2]:
CODE_ROOT = '.'

In [3]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
import random
ImageFile.LOAD_TRUNCATED_IMAGES = True
import argparse
import pickle
import sys
sys.path.append('%s/res/'%CODE_ROOT)
from models.models import get_model
from loader.loader import get_loader

/net/coxfs01/srv/export/coxfs01/pfister_lab2/share_root/Lab/spandan/federated_generalization
/net/coxfs01/srv/export/coxfs01/pfister_lab2/share_root/Lab/spandan/federated_generalization/./res/loader/loader.py
/net/coxfs01/srv/export/coxfs01/pfister_lab2/share_root/Lab/spandan/federated_generalization/./res/loader


In [4]:
from tqdm.notebook import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("poster")
sns.set_palette("Set1", 8, .75)
sns.despine()

<Figure size 432x288 with 0 Axes>

In [5]:
DATASET_NAMES = ['mnist_rotation_one_by_nine', 'mnist_rotation_three_by_nine',
                 'mnist_rotation_six_by_nine', 'mnist_rotation_nine_by_nine']
NUM_EPOCHS = 10
BATCH_SIZE = 200
ARCH = 'LATE_BRANCHING_COMBINED'

image_transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])


GPU = 1

In [6]:
NUM_CLASSES = (10,10,10,10)
loader_new = get_loader('multi_attribute_loader_file_list_mnist_rotation')

file_list_root = '%s/dataset_lists/mnist_rotation_lists/'%CODE_ROOT
att_path = '%s/dataset_lists/combined_attributes.p'%CODE_ROOT

In [7]:
shuffles = {'train':True,'val':True,'test':False}

data_dir = '%s/data/'%CODE_ROOT

all_dsets = {}
all_dset_loaders = {}
all_dset_sizes = {}

for DATASET_NAME in DATASET_NAMES:
    file_lists = {}
    dsets = {}
    dset_loaders = {}
    dset_sizes = {}
    for phase in ['train','val','test']:
        file_lists[phase] = "%s/%s_list_%s.txt"%(file_list_root,phase,DATASET_NAME)
        dsets[phase] = loader_new(file_lists[phase],att_path, image_transform, data_dir)
        dset_loaders[phase] = torch.utils.data.DataLoader(dsets[phase], batch_size=BATCH_SIZE, shuffle = shuffles[phase], num_workers=2,drop_last=True)
        dset_sizes[phase] = len(dsets[phase])
    all_dsets[DATASET_NAME] = dsets
    all_dset_loaders[DATASET_NAME] = dset_loaders
    all_dset_sizes[DATASET_NAME] = dset_sizes

/n/home05/smadan/.conda/envs/domain_adaptation/lib/python3.8/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [9]:
for data in dset_loaders['val']:
    break

In [10]:
len(data)

3

In [11]:
data[0].shape

torch.Size([200, 3, 28, 28])

In [12]:
data[1].shape

torch.Size([200, 4])

In [32]:
import torch

In [33]:
from torch.nn import functional as F

In [34]:
# data[1]

In [39]:
class Net(nn.Module):
    def __init__(self, num_gpus=0):
        super(Net, self).__init__()
        print(f"Using {num_gpus} GPUs to train")
        self.num_gpus = num_gpus
        device = torch.device(
            "cuda:0" if torch.cuda.is_available() and self.num_gpus > 0 else "cpu")
        print(f"Putting first 2 convs on {str(device)}")
        # Put conv layers on the first cuda device, or CPU if no cuda device
        self.conv1 = nn.Conv2d(3, 28, 3, 1).to(device)
        self.conv2 = nn.Conv2d(28, 64, 3, 1).to(device)
        # Put rest of the network on the 2nd cuda device, if there is one
        if "cuda" in str(device) and num_gpus > 1:
            device = torch.device("cuda:1")
        '''
        The setup here only supports 0-2 gpus, can be extended if more available
        '''

        print(f"Putting rest of layers on {str(device)}")
        self.dropout1 = nn.Dropout2d(0.25).to(device)
        self.dropout2 = nn.Dropout2d(0.5).to(device)
        self.fc1 = nn.Linear(9216, 128).to(device)
        self.fc2 = nn.Linear(128, 10).to(device)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)

        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        # Move tensor to next device if necessary
        '''
        This is necessary because we need to make sure the tensor we operate need to be on the same device
        '''
        next_device = next(self.fc1.parameters()).device
        x = x.to(next_device)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [40]:
nnmod = Net()

Using 0 GPUs to train
Putting first 2 convs on cpu
Putting rest of layers on cpu


In [47]:
data[1]

tensor([[0, 5, 0, 9],
        [0, 4, 0, 7],
        [0, 3, 0, 4],
        [0, 5, 0, 3],
        [0, 7, 0, 1],
        [0, 9, 0, 1],
        [0, 7, 0, 5],
        [0, 9, 0, 2],
        [0, 5, 0, 8],
        [0, 0, 0, 7],
        [0, 6, 0, 9],
        [0, 3, 0, 8],
        [0, 2, 0, 8],
        [0, 9, 0, 5],
        [0, 0, 0, 4],
        [0, 5, 0, 9],
        [0, 2, 0, 8],
        [0, 9, 0, 1],
        [0, 1, 0, 3],
        [0, 8, 0, 7],
        [0, 5, 0, 2],
        [0, 1, 0, 6],
        [0, 3, 0, 5],
        [0, 4, 0, 7],
        [0, 9, 0, 0],
        [0, 8, 0, 5],
        [0, 4, 0, 7],
        [0, 3, 0, 2],
        [0, 5, 0, 4],
        [0, 0, 0, 2],
        [0, 8, 0, 7],
        [0, 7, 0, 3],
        [0, 8, 0, 4],
        [0, 2, 0, 8],
        [0, 4, 0, 2],
        [0, 5, 0, 6],
        [0, 3, 0, 7],
        [0, 2, 0, 6],
        [0, 4, 0, 0],
        [0, 0, 0, 4],
        [0, 4, 0, 8],
        [0, 0, 0, 7],
        [0, 8, 0, 7],
        [0, 3, 0, 0],
        [0, 1, 0, 8],
        [0